# U.S. Geological Survey Class GW3099

Advanced Modeling of Groundwater Flow (GW3099) <br>
Boise, Idaho <br>
September 16 - 20, 2024 <br>

# ![title](../../images/ClassLocation.jpg)

# PRT example 1: backwards tracking in steady flow field

![title](../../images/ex-prt-mp7-p01-config.png)

This notebook demonstrates backwards tracking in a steady-state flow field. First a MODFLOW 6 PRT model and an equivalent MODPATH 7 model are run side-by-side, and their results compared. We then refine the grid around the release location and rerun the model to demonstrate the effects of a modified discretization.

Import dependencies.

In [6]:
import pathlib as pl
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvista as pv
import warnings
from flopy.export.vtk import Vtk
from flopy.utils.gridgen import Gridgen
from flopy.utils.gridintersect import GridIntersect
from flopy.discretization import VertexGrid
from flopy.utils.triangle import Triangle as Triangle
from flopy.utils.voronoi import VoronoiGrid
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from matplotlib import colormaps as cm

Ignore some warnings.

In [7]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)

Make sure executables are installed.

In [8]:
flopy.utils.get_modflow(":python", subset="mf6,mp7,gridgen,triangle")

auto-selecting option ':python' for 'bindir'
fetched release '19.0' info from MODFLOW-USGS/executables
using previous download '/Users/wbonelli/Downloads/modflow_executables-19.0-macarm.zip' (use 'force=True' to re-download)
extracting 4 files to '/Users/wbonelli/micromamba/envs/gw3099/bin'
gridgen (1.0.02) mf6 (6.5.0)      mp7 (7.2.001)    triangle (1.6)
updated flopy metadata file: '/Users/wbonelli/.local/share/flopy/get_modflow.json'


Create a base workspace.

In [9]:
example_name = "prt_mp7_ex2"
base_ws = pl.Path("temp") / example_name
base_ws.mkdir(exist_ok=True, parents=True)

## Flow model

First we define a flow model which will be used by PRT and MP7.

Define the flow model name and workspace.

In [10]:
gwf_name = f"{example_name}-gwf"
gwf_ws = base_ws / "gwf"
gwf_ws.mkdir(exist_ok=True, parents=True)

Define flow model units.

In [11]:
length_units = "feet"
time_units = "days"

Define flow model parameters.

In [12]:
nper = 1  # Number of periods
nlay = 3  # Number of layers (base grid)
nrow = 21  # Number of rows (base grid)
ncol = 20  # Number of columns (base grid)
delr = 500.0  # Column width ($ft$)
delc = 500.0  # Row width ($ft$)
top = 400.0  # Top of the model ($ft$)
botm = [220.0, 200.0, 0.0]  # Layer bottom elevations ($ft$)
porosity = 0.1  # Soil porosity (unitless)
rch = 0.005  # Recharge rate ($ft/d$)
kh = [50.0, 0.01, 200.0]  # Horizontal hydraulic conductivity ($ft/d$)
kv = [10.0, 0.01, 20.0]  # Vertical hydraulic conductivity ($ft/d$)
wel_q = -150000.0  # Well pumping rate ($ft^3/d$)
riv_h = 320.0  # River stage ($ft$)
riv_z = 317.0  # River bottom ($ft$)
riv_c = 1.0e5  # River conductance ($ft^2/d$)

Define the initial structured grid discretization.

In [13]:
Lx = 10000.0
Ly = 10500.0
nlay = 3
nrow = 21
ncol = 20
delr = Lx / ncol
delc = Ly / nrow
top = 400
botm = [220, 200, 0]

Define the time discretization.

In [14]:
nstp = 1
perlen = 1000.0
tsmult = 1.0
tdis_rc = [(perlen, nstp, tsmult)]

Construct a simulation for the flow model.

In [15]:
gwf_sim = flopy.mf6.MFSimulation(
    sim_name=gwf_name, exe_name="mf6", version="mf6", sim_ws=gwf_ws
)

Create the time discretization.

In [16]:
tdis = flopy.mf6.ModflowTdis(
    gwf_sim, pname="tdis", time_units="DAYS", perioddata=tdis_rc, nper=len(tdis_rc)
)

Create the flow model.

In [17]:
gwf = flopy.mf6.ModflowGwf(
    gwf_sim, modelname=gwf_name, model_nam_file="{}.nam".format(gwf_name)
)
gwf.name_file.save_flows = True

Create the discretization package.

In [18]:
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm
)

Create the initial conditions package.

In [19]:
ic = flopy.mf6.ModflowGwfic(gwf, pname="ic", strt=riv_h)  # initial heads at river stage

Create the node property flow package.

In [20]:
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    xt3doptions=[("xt3d")],
    icelltype=[1, 0, 0],
    k=kh,
    k33=kv,
    save_saturation=True,
    save_specific_discharge=True,
)

Define the model's boundary conditions. These include a well, a river, and recharge.

In [ ]:
# Well package
wel_loc = (2, 10, 9)
wd = [(wel_loc, wel_q)]

# River package
riv_iface = 6
riv_iflowface = -1
rd = []
for i in range(nrow):
    rd.append([(0, i, ncol - 1), riv_h, riv_c, riv_z, riv_iface, riv_iflowface])

# Recharge package
rch_iface = 6
rch_iflowface = -1

Define well and river cell numbers, used to extract and plot model results later.

In [ ]:
nodes = {}
k, i, j = wel_loc
nodes["well"] = [ncol * (nrow * k + i) + j]
nodes["river"] = []
for rivspec in rd:
    k, i, j = rivspec[0]
    node = ncol * (nrow * k + i) + j
    nodes["river"].append(node)

Create the recharge, well, and river packages.

In [ ]:
# Instantiate the MODFLOW 6 gwf recharge package
flopy.mf6.modflow.mfgwfrcha.ModflowGwfrcha(
    gwf,
    recharge=rch,
    auxiliary=["iface", "iflowface"],
    aux=[rch_iface, rch_iflowface],
)

# Instantiate the MODFLOW 6 gwf well package
flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(
    gwf, maxbound=1, stress_period_data={0: wd}
)

# Instantiate the MODFLOW 6 gwf river package
flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(
    gwf, auxiliary=["iface", "iflowface"], stress_period_data={0: rd}
)

Create the output control package.

In [ ]:
# Define output file names
headfile = f"{gwf_name}.hds"
budgetfile = f"{gwf_name}.cbb"

# Create package
head_record = [headfile]
budget_record = [budgetfile]
saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(
    gwf,
    pname="oc",
    saverecord=saverecord,
    head_filerecord=head_record,
    budget_filerecord=budget_record,
)

Create the solver package.

In [ ]:
ims = flopy.mf6.ModflowIms(
    gwf_sim,
    pname="ims",
    complexity="SIMPLE",
    outer_dvclose=1e-6,
    inner_dvclose=1e-6,
    rcloserecord=1e-6,
)

# ## Tracking models
# 
# We now define a MODPATH 7 particle tracking simulation, then an equivalent PRT simulation.

# ### PRT model

# Define the PRT model name and workspace.

In [ ]:
prt_name = f"{example_name}-prt"
prt_ws = base_ws / "prt"
prt_ws.mkdir(exist_ok=True, parents=True)

# Create a PRT simulation.

In [ ]:
prt_sim = flopy.mf6.MFSimulation(
    sim_name=prt_name, exe_name="mf6", version="mf6", sim_ws=prt_ws
)

# Create the temporal discretization.

In [ ]:
flopy.mf6.modflow.mftdis.ModflowTdis(
    prt_sim,
    pname="tdis",
    time_units="DAYS",
    nper=nper,
    perioddata=[(perlen, nstp, tsmult)],
)

# Create the PRT model.

In [ ]:
prt = flopy.mf6.ModflowPrt(
    prt_sim, modelname=prt_name, model_nam_file=f"{prt_name}.nam"
)

# Create the discretization package.

In [ ]:
flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(
    prt,
    pname="dis",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    length_units="FEET",
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

Create the PRT model input package.

In [ ]:
# Instantiate the MODFLOW 6 prt model input package.
# Assign a different zone number to active cells, well cells, and river cells.
# This makes it easier to determine where particles terminate.
izone = np.zeros((nlay, nrow, ncol), dtype=int)
for l, r, c in gwf.modelgrid.get_lrc(nodes["well"]):
    izone[l, r, c] = 1
for l, r, c in gwf.modelgrid.get_lrc(nodes["river"]):
    izone[l, r, c] = 2
flopy.mf6.ModflowPrtmip(prt, pname="mip", porosity=porosity, izone=izone)


Create the particle release package.

In [ ]:
# TODO: PRP

Create the output control package.

In [ ]:
# Define output files.
budgetfile_prt = f"{prt_name}.cbb"
trackfile_prt = f"{prt_name}.trk"
trackhdrfile_prt = f"{prt_name}.trk.hdr"
trackcsvfile_prt = f"{prt_name}.trk.csv"

# Define the output control package.
budget_record = [budgetfile_prt]
track_record = [trackfile_prt]
trackcsv_record = [trackcsvfile_prt]
tracktimes = list(range(0, 72000, 1000))
flopy.mf6.ModflowPrtoc(
    prt,
    pname="oc",
    budget_filerecord=budget_record,
    track_filerecord=track_record,
    trackcsv_filerecord=trackcsv_record,
    ntracktimes=len(tracktimes),
    tracktimes=[(t,) for t in tracktimes],
    saverecord=[("BUDGET", "ALL")],
)

Create the flow model interface.

In [ ]:
pd = [
    ("GWFHEAD", pl.Path(f"../{gwf_ws.name}/{headfile}")),
    ("GWFBUDGET", pl.Path(f"../{gwf_ws.name}/{budgetfile}")),
]
flopy.mf6.ModflowPrtfmi(prt, packagedata=pd)

Create an explicit model solution for the PRT model.

In [ ]:
ems = flopy.mf6.ModflowEms(
    prt_sim,
    pname="ems",
    filename=f"{prt_name}.ems",
)
prt_sim.register_solution_package(ems, [prt.name])

In [ ]:
# ### MP7 model
#
# Create a MODPATH 7 model.

## Grid refinement

We can now refine the grid around the particle release location.

First create a GRIDGEN object wrapping the flow model's grid, to which we'll add refinements.

In [ ]:
gridgen_ws = base_ws / "gridgen"
gridgen_ws.mkdir(parents=True, exist_ok=True)
gridgen = Gridgen(gwf.modelgrid, model_ws=gridgen_ws)

# Define refinement polygons.

In [ ]:
ref_polys = [
    [[(3500, 4000), (3500, 6500), (6000, 6500), (6000, 4000), (3500, 4000)]],  # outer
    [[(4000, 4500), (4000, 6000), (5500, 6000), (5500, 4500), (4000, 4500)]],  # middle
    [[(4500, 5000), (4500, 5500), (5000, 5500), (5000, 5000), (4500, 5000)]],  # inner
]
ref_paths = []
for i, poly in enumerate(ref_polys):
    gridgen.add_refinement_features([poly], "polygon", i + 1, range(nlay))
    ref_paths.append(gridgen_ws / f"rf{i}")

Build the refined grid.

In [ ]:
gridgen.build(verbose=False)
grid_props = gridgen.get_gridprops_vertexgrid()
disv_props = gridgen.get_gridprops_disv()
grid = flopy.discretization.VertexGrid(**grid_props)

Extract refined grid properties.

In [ ]:
ncpl = disv_props["ncpl"]
top = disv_props["top"]
botm = disv_props["botm"]
nvert = disv_props["nvert"]
vertices = disv_props["vertices"]
cell2d = disv_props["cell2d"]

Plot the grid with refinement levels superimposed.

In [ ]:
grid.plot()
for i, path in enumerate(ref_paths):
    flopy.plot.plot_shapefile(path, ax=plt.gca(), facecolor="green", edgecolor="none", alpha=(i + 1) / 3)
plt.show()

We can now modify the flow model simulation above:

1. Detach the structured grid discretization
2. Attach the refined grid discretization
3. Add the particle tracking model
4. Add a GWF-PRT model exchange

In [ ]:
sim = gwf_sim
sim.remove_package("dis")
disv = flopy.mf6.ModflowGwfdisv(
    gwf,
    length_units=length_units,
    **disv_props,  # from grid refinement above
)

Define the model's boundary conditions. These include a well, a river, and recharge. Instead of manually setting cell IDs for the well and river, we will determine cell IDs by defining boundary coordinates and intersecting them with the grid.

In [21]:
ix = GridIntersect(gwf.modelgrid, method="vertex", rtree=True)

Create the well package.

In [23]:
wel_coords = [(4718.45, 5281.25)]
welcells = ix.intersects(MultiPoint(wel_coords))
welcells = [icpl for (icpl,) in welcells]
welspd = [[(2, icpl), -150000.0] for icpl in welcells]
wel = flopy.mf6.ModflowGwfwel(gwf, print_input=True, stress_period_data=welspd)

Create the river package.

In [24]:
riv_iface = 6
riv_iflowface = -1
riverline = [(Lx - 1.0, Ly), (Lx - 1.0, 0.0)]
rivcells = ix.intersects(LineString(riverline))
rivcells = [icpl for (icpl,) in rivcells]
rivspd = [
    [(0, icpl), riv_h, riv_c, riv_z, riv_iface, riv_iflowface] for icpl in rivcells
]
riv = flopy.mf6.ModflowGwfriv(
    gwf, stress_period_data=rivspd, auxiliary=[("iface", "iflowface")]
)

IndexError: tuple index out of range

Create the recharge package.

In [ ]:
rch_iface = 6
rch_iflowface = -1
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=rch,
    auxiliary=["iface", "iflowface"],
    aux=[rch_iface, rch_iflowface],
)

Create the output control package.

In [ ]:
headfile_name = "{}.hds".format(gwf_name)
budgetfile_name = "{}.cbb".format(gwf_name)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    pname="oc",
    budget_filerecord=[budgetfile_name],
    head_filerecord=[headfile_name],
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

Create the iterative model solution (IMS) package and register it with the model.

In [ ]:
ims = flopy.mf6.ModflowIms(
    sim,
    pname="ims",
    outer_dvclose=1.0e-5,
    outer_maximum=100,
    under_relaxation="NONE",
    inner_maximum=100,
    inner_dvclose=1.0e-6,
    rcloserecord=0.1,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=0.99,
)
sim.register_ims_package(ims, [gwf.name])

Write and run the flow model.

In [ ]:
sim.write_simulation(silent=False)
sim.run_simulation(silent=False)

Load heads.

In [ ]:
hds = gwf.output.head().get_data()

Plot heads.

In [ ]:
fig = plt.figure(figsize=(5, 5))
fig.tight_layout()
ax = fig.add_subplot(1, 1, 1, aspect="equal")
mm = flopy.plot.PlotMapView(gwf, ax=ax, layer=2)
mm.plot_grid(alpha=0.25)
mm.plot_bc("WEL", plotAll=True, color="red")
mm.plot_bc("RIV", plotAll=True, color="teal")
pc = mm.plot_array(hds[:, 0, :], edgecolor="black", alpha=0.5)
cb = plt.colorbar(pc, shrink=0.25, pad=0.1)
cb.ax.set_xlabel(r"Head ($ft$)")
plt.show()